In [12]:
import sys

import onnx
import torch

sys.path.append("ddffnet")
from ddffnet.model.ours.PDNet_resnest import PDnet_ResNest_4F_SL_LA

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
state = torch.load("checkpoints/D-DFFNet.pth", map_location=device)

In [22]:
model = PDnet_ResNest_4F_SL_LA()
model.to(device)
model.load_state_dict(state)
model.eval()

model:PDnet_ResNest_4F_SL_LA


PDnet_ResNest_4F_SL_LA(
  (resnest): ResNest101(
    (conv1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

In [26]:
with torch.no_grad():
	x = torch.randn(1, 3, 320, 320).to(device)
	
	print("Exporting")
	onnx_program = torch.onnx.dynamo_export(model, x)
	
	print("Saving")
	onnx_program.save("checkpoints/D-DFFNet.onnx")

Exporting


/home/artem/Projects/blur_filtering/.venv-ddffnet/lib/python3.10/site-packages/torch/onnx/_internal/exporter.py:136: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(
/home/artem/Projects/blur_filtering/.venv-ddffnet/lib/python3.10/site-packages/torch/onnx/_internal/fx/passes/readability.py:53: UserWarning: Attempted to insert a get_attr Node with no underlying reference in the owning GraphModule! Call GraphModule.add_submodule to add the necessary submodule, GraphModule.add_parameter to add the necessary Parameter, or nn.Module.register_buffer to add the necessary buffer
  new_node = self.module.graph.get_attr(normalized_name)
/home/artem/Projects/blur_filtering/.venv-ddffnet/lib/python3.10/site-packages/torch/fx/graph.py:1460: UserWarning: Node resnest_conv1_1_running_mean target resnest/conv1/1/running_mean resnest/conv1/1/running_mean of  does n

Saving


In [27]:
print("Loading and validating")
onnx_model = onnx.load("checkpoints/D-DFFNet.onnx")
onnx.checker.check_model(onnx_model)
print("Everything is fine")

Loading and validating
Everything is fine
